In [ ]:
!gsutil cp -r gs://llama-mlhc/mistral-7b

Copying gs://llama-mlhc/mistral-7b/README.md...
Copying gs://llama-mlhc/mistral-7b/config.json...
Copying gs://llama-mlhc/mistral-7b/generation_config.json...
/ [3 files][  4.4 KiB/  4.4 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://llama-mlhc/mistral-7b/gitattributes...
Copying gs://llama-mlhc/mistral-7b/model-00001-of-00003.safetensors...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://llama-mlhc/mistral-7b/model-00002-of-00003.safetensors...
Copying gs://llama-mlhc/mistral-7b/model-00003-of-00003.safetensors..

In [ ]:
!pip install accelerate
!pip install bitsandbytes

In [ ]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    TrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForLanguageModeling,
    DataCollatorForSeq2Seq,
    AutoModelForCausalLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,


)
import json
import os
import argparse
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
import torch

In [ ]:
model = AutoModelForCausalLM.from_pretrained("mistral-7b/",
                                             load_in_4bit=False,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("mistral-7b/")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
prompts = []
for i in range(1000):
  prompt = """You are helping me study for the usmle.
              Share a low-severity example with two labels
              'symptoms:' and 'medical test result:'
              Do not share anything outside these labels"""
  inputs = tokenizer(prompt, return_tensors="pt")
  inputs = inputs.to("cuda")
  generate_ids = model.generate(**inputs, max_new_tokens=100, do_sample=True)
  new_sample = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  prompts.append(new_sample[0])

In [ ]:
pd.DataFrame({"prompts":prompts}).to_csv("low_severity_baseline_prompts.csv")

In [ ]:
prompts = []
for i in range(1000):
  prompt = """You are helping me study for the usmle.
              Share a high-severity example with two labels
              'symptoms:' and 'medical test result:'
              Do not share anything outside these labels"""
  inputs = tokenizer(prompt, return_tensors="pt")
  inputs = inputs.to("cuda")
  generate_ids = model.generate(**inputs, max_new_tokens=100, do_sample=True)
  new_sample = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  prompts.append(new_sample[0])

In [ ]:
pd.DataFrame({"prompts":prompts}).to_csv("high_severity_baseline_prompts.csv")

In [ ]:
prompts = pd.read_csv("baseline_prompts.csv")

In [ ]:
generated_id_list = []
for prompt in prompts["baseline_prompt_score"].tolist():
  inputs = tokenizer(prompt, return_tensors="pt")
  inputs = inputs.to("cuda")
  generate_ids = model.generate(**inputs, max_new_tokens=15, do_sample=True)
  new_sample = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  generated_id_list.append(new_sample[0])

In [ ]:
pd.DataFrame({"prompts":generated_id_list}).to_csv("baseline_scores_mlhc.csv")